In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

sns.set(style = "whitegrid")

'''
sdgksjgbsd
'''

'\nsdgksjgbsd\n'

In [2]:
class NaiveBayesClassifier():
  '''
  Bayes Theorem form
  P(y|X) = P(X|y)*P(y)/P(X)

  '''
  def calc_prior(self, features, target):
    '''
    prior probability P(y)
    '''
    self.prior = (features.groupby(target).apply(lambda x: len(x))/self.rows).to_numpy()
    return self.prior
  

  def calc_stats(self, features, target):
    '''
    calc mean, variance  for each column
    '''
    self.mean = features.groupby(target).apply(np.mean).to_numpy()
    self.var = features.groupby(target).apply(np.var).to_numpy()
    return self.mean, self.var

  def gaussian_den(self, class_idx, x):


    # calculate the prob from the gaussian density function (normal dis)

    mean = self.mean[class_idx]
    var = self.var[class_idx]
    numerator = np.exp((-1/2)*((x-mean)**2)/(2*var))
    den = np.sqrt(2* np.pi *var)
    prob = numerator/den
    return prob


  def calc_posterior(self, x):

    #calculate the posterior prob for each class
    posterior = []
    for i in range(self.count):
      prior = np.log(self.prior[i]) # numerically stable by utilizing the log function

      conditional = np.sum(np.log(self.gaussian_den(i, x)))

      post = prior + conditional
      posterior.append(post)
    
    return self.class_target[np.argmax(posterior)]


  def fit(self, features, target):

    self.class_target = np.unique(target)

    self.count = len(self.class_target)

    self.feature_num = features.shape[0]

    self.rows = features.shape[0]

    self.calc_stats(features, target)
    self.calc_prior(features, target)


  def predict(self, features):
    predictions = [self.calc_posterior(f) for f in features.to_numpy()]
    return predictions
  
  def accuracy(self, y_test, y_pred):
    accuracy = np.sum(y_test == y_pred)/len(y_test)
    return accuracy 

  

  

In [3]:
reviews = []
sentiment = []
with open('customerReviews.txt', 'r') as file:
    for line in file:
        review, label = line.rsplit(',', 1)
        reviews.append(review)
        sentiment.append(label.strip())

#print(reviews)
#print(sentiment)
reviews = np.array(reviews)
sentiment = np.array(sentiment)

X_train, X_test, y_train, y_test = train_test_split(reviews, sentiment, test_size = 0.2, random_state = 123)
print(X_train.shape, y_train.shape)

X_train_df = pd.DataFrame(X_train, columns=['Review'])
y_train_df = pd.DataFrame(y_train, columns=['Sentiment'])
X_test_df = pd.DataFrame(X_test, columns=['Review'])
y_test_df = pd.DataFrame(y_test, columns=['Sentiment'])
NB = NaiveBayesClassifier()

NB.fit(X_train_df, y_train_df['Sentiment'])  # pass the target column only

predictions = NB.predict(X_test_df)
accuracy = NB.accuracy(y_test_df['Sentiment'], predictions)

print(f'Accuracy: {accuracy:.4f}')

(112,) (112,)
Accuracy: 0.5172


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3755: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [4]:
new_data = pd.DataFrame({'review': ['I had a terrible experience with this company', 
                                    'This is a great company with excellent customer service', 
                                    'I was really disappointed with this product', 
                                    'The service is too expensive for what it offers',]})

predictions = NB.predict(new_data)

i = 0
for reviews in new_data['review']:

  print(new_data['review'][i] + ". Prediction: " + predictions[i])
  i = i +1

I had a terrible experience with this company. Prediction: negative
This is a great company with excellent customer service. Prediction: negative
I was really disappointed with this product. Prediction: negative
The service is too expensive for what it offers. Prediction: negative
